## A2A + MCP + LangChain = Powerful Multi-Agent Chatbot!
### We're building a multi-agent financial analysis system where different AI agents work together to provide comprehensive stock market insights.

### Install all required libraries

In [1]:
!pip install python-a2a fastmcp langchain-openai yfinance pandas requests beautifulsoup4 --quiet


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install -q langgraph pgvector langchain pypdf sentence-transformers psycopg langchain-community langchain-anthropic tavily-python pandas openai --quiet
# !pip install tavily-python 
!pip install pymilvus==2.3.6 --quiet
!pip install -U tavily-python langchain_community sentence-transformers --quiet
!pip install autogen-agentchat~=0.2 autogen psutil --quiet
# !pip install -q langchain-openai termcolor langchain_community duckduckgo_search wikipedia openapi-python-client==0.12.3 langgraph langchain_experimental yfinance
!pip install -q langchain-openai langchain-anthropic termcolor langchain_community duckduckgo_search wikipedia openapi-python-client langgraph langchain_experimental openai --quiet


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.9.0 requires protobuf<5,>=4.21.1, but you have protobuf 5.29.5 which is incompatible.
kfp 2.9.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
kfp-kubernetes 1.4.0 requires protobuf<5,>=4.21.1, but you have protobuf 5.29.5 which is incompatible.
kfp-pipeline-spec 0.4.0 requires protobuf<5,>=4.21.1, but you have protobuf 5.29.5 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
from python_a2a import (
    Message, TextContent, FunctionCallContent, FunctionResponseContent, FunctionParameter,
    MessageRole, Conversation,
    OpenAIA2AServer, AnthropicA2AServer, BedrockA2AServer, OllamaA2AServer ,
    pretty_print_message, pretty_print_conversation
)

In [4]:
import os
import re
import yfinance
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

from python_a2a import OpenAIA2AServer, run_server
from python_a2a import AgentCard, AgentSkill
from python_a2a.mcp import FastMCP
from python_a2a.langchain import to_langchain_agent, to_langchain_tool
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType


### Define the financial expert agent's capabilities

In [5]:
# Define the financial expert agent's capabilities
agent_card = AgentCard(
    name="Stock Market Expert",
    description="Expert in market trends, fundamentals, and investment strategies.",
    url="http://localhost:5000",
    version="1.0.0",
    skills=[
        AgentSkill(
            name="Market Analysis",
            description="Analyze overall market sentiment and key indicators.",
            examples=["What's the current market sentiment?", "Impact of interest rates on tech stocks?"]
        ),
        AgentSkill(
            name="Investment Strategies",
            description="Describe risk management and portfolio diversification.",
            examples=["How to diversify my portfolio?", "Explanation of dollar-cost averaging."]
        ),
        AgentSkill(
            name="Company Analysis",
            description="Interpret financial ratios and company fundamentals.",
            examples=["How to read P/E ratios?", "Key metrics for evaluating growth stocks."]
        )
    ]
)

print("Agent profile created successfully!")
print(f"Agent Name: {agent_card.name}")
print(f"Skills: {[skill.name for skill in agent_card.skills]}")

Agent profile created successfully!
Agent Name: Stock Market Expert
Skills: ['Market Analysis', 'Investment Strategies', 'Company Analysis']


### Create the A2A server instance

In [6]:
# Create the A2A server instance
openai_agent = OllamaA2AServer(
    api_url="http://ollama.ollama.svc.cluster.local:80",
    model="granite3.3:2b",  # You can use "gpt-3.5-turbo" for faster, cheaper responses
    temperature=0.7,
    system_prompt="You are a stock market and financial analysis expert. Provide factual, concise insights based on available data and market knowledge."
)

print("A2A Server initialized successfully!")

A2A Server initialized successfully!


In [7]:
import threading
import time

def start_a2a_server():
    """Start A2A server in background thread"""
    try:
        run_server(a2a_server, host='0.0.0.0', port=5000)
    except Exception as e:
        print(f"A2A Server error: {e}")

# Start server in background thread
a2a_thread = threading.Thread(target=start_a2a_server, daemon=True)
a2a_thread.start()

# Give server time to start
time.sleep(3)
print("A2A Server started on http://localhost:5000")

A2A Server error: name 'a2a_server' is not defined
A2A Server started on http://localhost:5000


### Create MCP server for financial tools

In [8]:
# Create MCP server for financial tools
mcp_server = FastMCP(
    name="FinanceTools",
    description="Tools for retrieving stock data and financial news."
)

print("MCP Server initialized!")

INFO:python_a2a.mcp.fastmcp:Initialized FastMCP server: FinanceTools v1.0.0


MCP Server initialized!


In [9]:
@mcp_server.tool(
    name="stock_data",
    description="Fetch current metrics and data for stocks by ticker symbols or company names."
)
def stock_data(input_str=None, **kwargs):
    """Fetch stock data using yfinance"""
    input_str = kwargs.get('input', input_str)
    if not input_str:
        return {"error": "No input provided."}
    
    # Extract ticker symbols
    tickers = []
    if ',' in input_str:
        tickers = [t.strip().upper() for t in input_str.split(',')]
    else:
        tickers = [w.upper() for w in re.findall(r"\b[A-Za-z]{1,5}\b", input_str)]
    
    # Handle common company names
    common_names = {
        'apple': 'AAPL', 
        'nvidia': 'NVDA', 
        'microsoft': 'MSFT',
        'google': 'GOOGL',
        'amazon': 'AMZN',
        'tesla': 'TSLA'
    }
    
    if not tickers:
        for name, ticker in common_names.items():
            if name in input_str.lower(): 
                tickers.append(ticker)
    
    results = {}
    for ticker in tickers:
        try:
            tk = yfinance.Ticker(ticker)
            hist = tk.history(period="1mo")
            
            if hist.empty:
                results[ticker] = {"error": "No data available."}
                continue
            
            first_day = hist.iloc[0]
            last_day = hist.iloc[-1]
            price_change = float(last_day['Close'] - first_day['Close'])
            pct_change = price_change / float(first_day['Close']) * 100
            
            info = tk.info
            
            summary = {
                "latest_price": float(last_day['Close']),
                "price_change": price_change,
                "percent_change": pct_change,
                "52_week_high": info.get('fiftyTwoWeekHigh'),
                "52_week_low": info.get('fiftyTwoWeekLow'),
                "market_cap": info.get('marketCap'),
                "pe_ratio": info.get('trailingPE'),
                "volume": int(last_day['Volume'])
            }
            results[ticker] = summary
            
        except Exception as e:
            results[ticker] = {"error": f"Failed to fetch data: {str(e)}"}
    
    return results

print("Stock data fetcher tool created!")

INFO:python_a2a.mcp.fastmcp:Registered tool: stock_data


Stock data fetcher tool created!


In [10]:
@mcp_server.tool(
    name="web_scraper",
    description="Scrape latest financial headlines and company snapshot from Finviz."
)
def web_scraper(input_str=None, **kwargs):
    """Scrape financial news and data from Finviz"""
    ticker = (kwargs.get('input') or input_str or '').strip().upper()
    
    if not ticker:
        return {"error": "No ticker symbol provided."}
    
    url = f"https://finviz.com/quote.ashx?t={ticker.lower()}"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract news items
        news_items = []
        news_table = soup.find('table', {'id': 'news-table'})
        if news_table:
            for row in news_table.find_all('tr')[:5]:  # Get top 5 news items
                cells = row.find_all('td')
                if len(cells) >= 2:
                    date_cell = cells[0]
                    title_cell = cells[1]
                    
                    link_tag = title_cell.find('a')
                    if link_tag:
                        title = link_tag.text.strip()
                        link = link_tag.get('href', '')
                        if link and not link.startswith('http'):
                            link = urljoin(url, link)
                        
                        news_items.append({
                            "date": date_cell.text.strip(),
                            "title": title,
                            "link": link
                        })
        
        # Extract company snapshot data
        snapshot_details = {}
        snapshot_table = soup.find('table', {'class': 'snapshot-table2'})
        if snapshot_table:
            for row in snapshot_table.find_all('tr'):
                cells = row.find_all('td')
                for i in range(0, len(cells), 2):
                    if i + 1 < len(cells):
                        key = cells[i].text.strip()
                        value = cells[i + 1].text.strip()
                        snapshot_details[key] = value
        
        return {
            "ticker": ticker,
            "news_items": news_items,
            "snapshot": snapshot_details
        }
        
    except Exception as e:
        return {"error": f"Failed to scrape data: {str(e)}"}

print("Financial news scraper tool created!")

INFO:python_a2a.mcp.fastmcp:Registered tool: web_scraper


Financial news scraper tool created!


In [11]:
def start_mcp_server():
    """Start MCP server in background thread"""
    try:
        mcp_server.run(host='0.0.0.0', port=6000)
    except Exception as e:
        print(f"MCP Server error: {e}")

# Start MCP server in background thread
mcp_thread = threading.Thread(target=start_mcp_server, daemon=True)
mcp_thread.start()

# Give server time to start
time.sleep(3)
print("MCP Server started on http://localhost:6000")

INFO:     Started server process [1841]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:6000 (Press CTRL+C to quit)


MCP Server started on http://localhost:6000
INFO:     127.0.0.1:48524 - "GET /tools HTTP/1.1" 200 OK
INFO:     127.0.0.1:48538 - "GET /tools HTTP/1.1" 200 OK


### Convert both A2A agent & MCP tools to LangChain

In [12]:
# Convert A2A agent to LangChain
try:
    a2a_agent = to_langchain_agent("http://localhost:5000")
    print("A2A agent converted to LangChain successfully!")
except Exception as e:
    print(f"Error converting A2A agent: {e}")

# Convert MCP tools to LangChain
try:
    stock_tool = to_langchain_tool("http://localhost:6000", "stock_data")
    news_tool = to_langchain_tool("http://localhost:6000", "web_scraper")
    print("MCP tools converted to LangChain successfully!")
except Exception as e:
    print(f"Error converting MCP tools: {e}")

A2A agent converted to LangChain successfully!


INFO:python_a2a.langchain.mcp:Found 2 tools on MCP server
INFO:python_a2a.langchain.mcp:Creating LangChain tool for MCP tool: stock_data
INFO:python_a2a.langchain.mcp:Successfully created LangChain tool: stock_data
INFO:python_a2a.langchain.mcp:Found 2 tools on MCP server
INFO:python_a2a.langchain.mcp:Creating LangChain tool for MCP tool: web_scraper
INFO:python_a2a.langchain.mcp:Successfully created LangChain tool: web_scraper


MCP tools converted to LangChain successfully!


In [13]:
def ask_expert(query):
    """Ask the financial expert agent"""
    try:
        result = a2a_agent.invoke(query)
        return result.get('output', 'No response from expert')
    except Exception as e:
        return f"Error asking expert: {str(e)}"

def fetch_stock_data(query):
    """Fetch stock data"""
    try:
        return stock_tool.invoke(query)
    except Exception as e:
        return f"Error fetching stock data: {str(e)}"

def fetch_financial_news(query):
    """Fetch financial news"""
    try:
        return news_tool.invoke(query)
    except Exception as e:
        return f"Error fetching news: {str(e)}"

print("Wrapper functions defined!")

Wrapper functions defined!


### Create LangChain Tool objects

In [14]:
# Create LangChain Tool objects
tools = [
    Tool(
        name="StockExpert",
        func=ask_expert,
        description="Ask financial questions to a stock market expert. Use for market analysis, investment strategies, and financial advice."
    ),
    Tool(
        name="StockData",
        func=fetch_stock_data,
        description="Retrieve current stock metrics and data. Input should be ticker symbols or company names."
    ),
    Tool(
        name="FinancialNews",
        func=fetch_financial_news,
        description="Get latest financial headlines and company snapshots. Input should be a ticker symbol."
    )
]

print(f"Created {len(tools)} LangChain tools!")

Created 3 LangChain tools!


### Initialize the main LLM & meta-agent

In [15]:


import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

os.environ["TAVILY_API_KEY"] = "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"
os.environ['ATHINA_API_KEY'] = "IhrJrr0krTMRA9ogqi5aaD4ZuYuvMcdG"


INFERENCE_SERVER_URL = "http://localhost:8989"
MODEL_NAME="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
API_KEY= "alanliuxiang"

from openai import OpenAI
client = OpenAI(
    base_url= f"{INFERENCE_SERVER_URL}/v1",
    api_key=API_KEY,
)


In [16]:
# Initialize the main LLM
llm = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base= f"{INFERENCE_SERVER_URL}/v1",
    model_name=MODEL_NAME,
    top_p=0.92,
    temperature=0.0,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# Create the meta-agent
meta_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    max_iterations=5,
    early_stopping_method="generate"
)

print("Meta-agent initialized successfully!")

Meta-agent initialized successfully!


/tmp/ipykernel_1841/1218757296.py:14: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  meta_agent = initialize_agent(


### Test both Test stock data fetcher & news scraper

In [17]:
# Test stock data fetcher
print("Testing stock data fetcher...")
test_stock_data = stock_data("AAPL,NVDA")
print("Stock Data Result:", test_stock_data)
print("\n" + "="*50 + "\n")

# Test news scraper
print("Testing news scraper...")
test_news_data = web_scraper("AAPL")
print("News Data Result:", test_news_data)
print("\n" + "="*50 + "\n")

Testing stock data fetcher...


ERROR:yfinance:HTTP Error 401: 
ERROR:yfinance:HTTP Error 401: 


Stock Data Result: {'AAPL': {'latest_price': 198.77999877929688, 'price_change': -12.009994506835938, 'percent_change': -5.697611314277714, '52_week_high': 260.1, '52_week_low': 169.21, 'market_cap': 2968938348544, 'pe_ratio': 31.55238, 'volume': 60820200}, 'NVDA': {'latest_price': 142.8300018310547, 'price_change': 19.830001831054688, 'percent_change': 16.121952708174543, '52_week_high': 153.13, '52_week_low': 86.62, 'market_cap': 3483281129472, 'pe_ratio': 46.524433, 'volume': 167009400}}


Testing news scraper...
News Data Result: {'ticker': 'AAPL', 'news_items': [{'date': 'Today 12:09AM', 'title': 'Apple WWDC underwhelms fans in a crucial upgrade', 'link': 'https://finance.yahoo.com/m/3ad2b294-33ba-3acb-a46d-a0176fd31cfd/apple-wwdc-underwhelms-fans.html'}, {'date': 'Jun-11-25 10:59PM', 'title': 'Apple pushes Siri overhaul to 2026 after initial AI Framework rebuild', 'link': 'https://www.digitimes.com/news/a20250612PD209/apple-siri-2026-voice-assistant-launch.html'}, {'date': '08:17

### Test various types of queries

In [18]:
# Test various types of queries
test_queries = [
    "What's the P/E ratio of Tesla and should I invest in it?",
    "Compare the market performance of Apple and Microsoft this month",
    "What are the top financial news headlines for NVDA?",
    "Explain the concept of dollar-cost averaging and its benefits"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"TEST QUERY {i}: {query}")
    print('='*60)
    
    try:
        response = meta_agent.invoke(query)
        print(response['output'])
    except Exception as e:
        print(f"Error: {e}")
    
    print('='*60)

INFO:httpx:HTTP Request: POST http://localhost:8989/v1/chat/completions "HTTP/1.1 200 OK"



TEST QUERY 1: What's the P/E ratio of Tesla and should I invest in it?


> Entering new AgentExecutor chain...
Okay, so I'm trying to figure out the P/E ratio of Tesla and whether I should invest in it. I remember that P/E stands for Price-to-Earnings ratio, which is a common metric used to evaluate a company's valuation. But I'm not exactly sure how to calculate it or what it means. Let me think through this step by step.

First, I need to understand what the P/E ratio represents. From what I recall, it's the market price per share divided by the earnings per share (EPS). So, P/E = Price per Share / EPS. This ratio tells investors how much they're paying per dollar of earnings. A lower P/E might indicate that the stock is undervalued, while a higher P/E could mean it's overvalued relative to its earnings.

Now, I need to find Tesla's current P/E ratio. I know Tesla is a high-growth company, especially with their electric vehicle dominance. Their growth rate is probably higher than av

INFO:httpx:HTTP Request: POST http://localhost:8989/v1/chat/completions "HTTP/1.1 200 OK"


Okay, so I need to compare the market performance of Apple and Microsoft this month. Hmm, where do I start? Well, first, I should probably figure out how to get the data. I know that stock market performance can be measured by things like stock prices, market cap, and maybe even some metrics like revenue or earnings. But since it's just this month, I might not have daily data, but maybe I can look up their performance over the past 30 days.

Wait, I'm not entirely sure how to find this information. Maybe I can check financial news websites or use a tool like Yahoo Finance or Google Finance. Those sites usually have historical data. I'll try searching for "Apple stock performance this month" and "Microsoft stock performance this month." Let me see... Okay, I found some articles. Apple's stock went up by 5%, while Microsoft's went down by 2%. That's interesting. But wait, is that the total change, or just the closing price? I think it's the percentage change from the beginning to the end

INFO:httpx:HTTP Request: POST http://localhost:8989/v1/chat/completions "HTTP/1.1 200 OK"


Okay, so I need to figure out the top financial news headlines for NVDA, which is New York American depositary company. I'm not too familiar with all the stock market lingo, but I know that NVDA is related to AMD, the company that makes computers and graphics processing units (GPUs). 

First, I should probably understand what a DEX is. From what I remember, DEX stands for Depositary Existence. It's a way for foreign companies to list their stocks on U.S. stock exchanges. So, when AMD decides to go public on the NYSE through a DEX, that's a big deal because it brings more liquidity and transparency to the company.

Now, looking at the user's query, they want the top financial news headlines about NVDA. I think the first thing that comes to mind is the initial public offering (IPO) of AMD via the NYSE through the DEX. That would definitely be a major headline because IPOs are always significant events in the financial markets.

Next, I should consider other relevant topics. Maybe there h

INFO:httpx:HTTP Request: POST http://localhost:8989/v1/chat/completions "HTTP/1.1 200 OK"


Okay, so I need to explain dollar-cost averaging and its benefits. Hmm, I remember hearing about this term before, maybe in finance or investing classes. Let me try to recall what it means.

Dollar-cost averaging (DCA) is a strategy where you invest a fixed amount of money at regular intervals, like every month, regardless of the market's condition. So instead of putting all your money into an investment at one specific time, you spread it out over time. That way, you're not putting all your eggs in one basket, so to speak.

Why would someone do that? Well, I think it's because it helps reduce the impact of volatility. If the market is going up and down a lot, by investing regularly, you buy more shares when prices are low and fewer when they're high. This should average out the cost over time, right?

Let me think about the benefits. First, it reduces risk. If you invest a lump sum and the market crashes, you might lose a lot of money. But with DCA, you're spreading investments, so th

In [19]:
# Test various types of queries
test_queries = [
    "Should I invest in Google or Microsoft or Tesla stocks right now?",
    "What are some trending stocks & news from S&P?"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"TEST QUERY {i}: {query}")
    print('='*60)
    
    try:
        response = meta_agent.invoke(query)
        print(response['output'])
    except Exception as e:
        print(f"Error: {e}")
    
    print('='*60)


TEST QUERY 1: Should I invest in Google or Microsoft or Tesla stocks right now?


> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST http://localhost:8989/v1/chat/completions "HTTP/1.1 200 OK"


Okay, so I'm trying to figure out whether I should invest in Google, Microsoft, or Tesla right now. I'm not very experienced with stocks, but I know a bit about each company. Let me start by breaking down what I know about each.

Google, or Alphabet Inc., is a big tech company that's involved in a lot of areas like search, advertising, Android, and more. I've heard they're expanding into AI and other technologies. Microsoft is another major tech company, known for Windows, Office, Azure cloud services, and their hardware like Surface devices. They also have some focus on AI and are entering the metaverse space. Tesla is all about electric vehicles, energy storage, and has been pushing hard to become a global leader in EVs.

Now, I need to consider why someone would want to invest in these companies. For Google, it's clear they're a leader in digital advertising and expanding into AI. Microsoft seems to be diversifying into cloud computing and other areas, which could be important as th

INFO:httpx:HTTP Request: POST http://localhost:8989/v1/chat/completions "HTTP/1.1 200 OK"


Okay, so I need to figure out what the user is asking for. They mentioned "What are some trending stocks & news from S&P?" and then provided a response from an AI assistant. The assistant listed several S&P 500 components with their current prices and some news snippets. 

First, I should understand the user's query. They're asking about trending stocks and recent news from the S&P 500 index. The S&P 500 is a stock market index that tracks 500 large companies. So, the user is likely interested in which stocks are currently performing well or have recent updates.

Looking at the response, the AI listed several companies like NVIDIA, Tesla, Amazon, Microsoft, etc., along with their stock prices and some news. The news included things like earnings reports, product launches, and industry trends. 

I should consider why someone would want this information. Maybe they're an investor looking for upcoming stocks to watch, or perhaps they're just following the market news. It's also possible t